
# Contents of This Note Book
* Importing Libraries and Data Set
* Preprocessing
* Model Creation & Training
* Saving the Model and its Weights
* Model Prediction / Recognition
* TF Model to TF Lite Conversion
* TF Lite Interpreter
* TF Lite Model Prediction

**Importing Libraries**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

**Importing Data Set**

In [ ]:
# load the drive helper and mount
from google.colab import drive

# this will prompt for authorization.
drive.mount('/content/drive')

# after executing the cell above, drive files will be present in
# "/content/drive/My Drive"

!ls "/content/drive/My Drive"

os.chdir("/content/drive/My Drive/dataset_new")

**Preprocessing**

In [ ]:
# normalization
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

# preparing data set
training_set = train_datagen.flow_from_directory(
                                    '/content/drive/MyDrive/dataset_new/training set',
                                                 target_size = (128, 128),
                                                 color_mode = 'grayscale',
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(
                                    '/content/drive/MyDrive/dataset_new/test set',
                                            target_size = (128, 128),
                                            color_mode = 'grayscale',
                                            batch_size = 32,
                                            class_mode = 'categorical')

**Model Creation**

In [ ]:
# initialising the CNN
classifier = Sequential()

# convolution to extract features from images
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 1), activation = 'relu'))

# max pooling to get max / largest values in feature map
# down sampling technique to get the most present features
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# more convolution and max pooling layers
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(256, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# flattening is converting the data into a 1-dimensional array
classifier.add(Flatten())
classifier.add(Dense(units = 1024, activation = 'relu'))
classifier.add(Dense(units = 14, activation = 'softmax'))

# compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# show summary of the created model
classifier.summary()

**Model Training**

In [ ]:
# less epochs because we have large number of data set and data set
# is closely related so the model will learn it faster
classifier.fit(training_set,
               steps_per_epoch = 490,
               epochs = 2,
               validation_data = test_set,
               validation_steps = 19)

**Saving Model & Weights**

In [ ]:
# saving model and weights which can be used later without training 
classifier.save("/content/cal_2e2.h5")
classifier.save_weights("/content/cal_weights_2e2.hdf5")

**Recognition / Prediction**

In [ ]:
# test images are manually upload from the 'files' tab on the left side
# classification models take much time to train, so we can train them one time 
#   and can load them for prediction
# ***load the model first for recognition directly*** 

test_image = image.load_img('/content/ten1424.jpg', target_size = (128, 128), color_mode='grayscale')
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
tf_prediction = np.argmax(result)
print ('TF prediction:', tf_prediction)

# printing model size in mb
print ('TF Size:', round(os.path.getsize('/content/cal_2e2.h5')/(1024*1024), 3) , 'MB')

# TF Lite Started

**TF Model to TF Lite Conversion**

In [ ]:
# convert & optimize the model
converter = tf.lite.TFLiteConverter.from_keras_model(classifier)
converter.optimization = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

**TF Lite Model Saving & Size Check**

In [ ]:
# save the model
open('tflite_model.tflite', 'wb').write(tflite_model)

# model size check
print ('TF Lite Size:', round(os.path.getsize('tflite_model.tflite')/(1024), 3) , 'KB')

**Interpreter**

In [ ]:
# 'interpreter' to interpret tf lite model and to get input & output details
# models obtained from 'TfLiteConverter' can be run in python with 'interpreter'
interpreter = tf.lite.Interpreter('tflite_model.tflite')

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# print input & output values & shapes to check everything is same 
# as the orignal tf model.
# data types etc are changed during tf lite conversion & optimizations
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

**Prediction from TF Lite Model**

In [ ]:
testx = image.load_img('/content/three1154.jpg', target_size = (128, 128), color_mode='grayscale')
testx = image.img_to_array(testx)
testx = np.expand_dims(testx, axis = 0)

# needed before execution
# tensorFlow lite pre-plans tensor allocations to optimize inference,
# so the user needs to call allocate_tensors() before any inference.
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], testx)
interpreter.invoke()

# predicting from tf lite model
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

# getting max index 
tf_lite_prediction = np.argmax(tflite_model_predictions)
print ('TF Lite Prediction : ',tf_lite_prediction)